### 数据集介绍：

#### covid.train.csv

文件中包含了94列的数据，其中前40列数据表示美国的州，其后的每18列作为一天内产生的数据。这18列中，前17列是被调查者的实际情况，第18列是测试为阳性的比例。

#### covid.test.csv

文件形式与前面的训练数据相同，不同的是第三天的第18列（测试为阳性的比例）需要通过模型来预测，这也是本次作业的目的。

In [2]:
# 这一模块需要引入的库：
import csv
import numpy
import tensor
from tensor.utils.data import Dataset, DataLoader

class Covid19dataset(Dataset):
    def __init__(self, path, mode='train'):
        super().__init__()
        self.mode = mode

        # 读取数据
        with open(path) as file:
            data_csv = list(csv.reader(file))   # 将csv文件中获取的数据转换为列表类型
            data = np.array(data_csv[1:])[:, 1:].astype(float)  # 获取文件中的【数值数据】

        if mode == 'test':  # 由于训练集、验证集与测试集的数据有所不同（最后一列数据），在此分情况运行
            data = data[:, 0:93]    # 测试集中，由于最后一行是得预测的数据，故只有93列
            self.data = torch.FloatTensor(data)     # 将numpy类型数据转换为tensor类型
        else:
            target = data[:, -1]    # 训练集和验证集中用于对比结果的“目标”
            data = data[:, 0:93]
            train_index = []
            dev_index = []
            for i in range(data.shape[0]):     # 这个循环用于将covid.train.csv文件中的数据分为训练集和验证集
                if i % 10 != 0:                # 取序号为整十数的样本作为验证集
                    train_index.append(i)
                else:
                    dev_index.append(i)
            if mode == 'train':     # 训练集的数据
                self.target = torch.FloatTensor(target[train_index])
                self.data = torch.FloatTensor(data[train_index, 0:93])
            else:       # 测试集的数据
                self.target = torch.FloatTensor(target[dev_index])
                self.data = torch.FloatTensor(data[dev_index, 0:93])

        # 此处是对数据进行标准化处理，可以将不同量纲的不同特征，变为同一个数量级，使得损失函数更加平滑
        # 标准化的优点：①提升模型的精度   ②提升收敛速度
        # 采用均值标准化： （第i维数据 - 第i维数据的平均值）/（第i维数据的标准差）
        self.data[:, 40:] = (self.data[:, 40:] - self.data[:, 40:].mean(dim=0)) / self.data[:, 40:].std(dim=0)

        self.dim = self.data.shape[1]   # 获取数据的列数

    def __getitem__(self, item):        # 【注】这是Dataset必须重写的类函数，意在按索引返回数据
        if self.mode == 'train' or self.mode == 'dev':  # 训练集和验证集包含特征和目标数据
            return self.data[item], self.target[item]
        else:
            return self.data[item]      # 测试集仅含有特征数据

    def __len__(self):                  # 【注】返回数据的行数
        return len(self.data)


def prep_dataloader(path, mode, batch_size, n_jobs=0):
    dataset = Covid19dataset(path, mode)    # 定义一个Dataset类
    dataloader = DataLoader(dataset, batch_size,
                            shuffle=(mode == 'train'),  # 【是否打乱数据后再读取】
                            drop_last=False,            # 【False表示不丢弃不能被batch_size整除的部分】
                            num_workers=n_jobs,         # 【采用多少个进程读取数据】
                            pin_memory=False)           # 【是否将数据载入CUDA的内存当中】
    print(mode, 'data done!')
    return dataloader

2024-03-09 00:30:06.146175: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 00:30:06.146229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 00:30:06.147955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-09 00:30:06.158387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 00:30:07.498703: W tensorflow/compiler/tf2

ModuleNotFoundError: No module named 'tensor'